In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/beans/bespoke')

from constants import *
from imports import *

In [2]:
for i in range(100):
    paths = glob.glob(f"{BLENDER_MEMBANK_ROOT}/**/obs_per_sec.npy", recursive=True)
    obs_per_sec = np.array([np.load(p)[0] for p in paths])
    print(obs_per_sec)
    time.sleep(60*1)

[]
[]
[10.056406]
[10.056406   8.140766   6.8471203  8.033258   8.767206   7.019197
  6.665913   6.6422606]
[10.056406   8.140766   6.8471203  5.9354215  6.2444777  8.033258
 11.055006   6.146068   7.019197   6.665913   6.6422606  6.0954986]
[ 7.875688  11.704005  13.745807   5.9354215  6.2444777  8.033258
 11.055006   6.146068   7.019197   6.665913  14.838375   6.0954986]
[]
[13.018165 14.862092 13.642484 15.079833 14.061663 13.063938 14.189661]
[12.192576 13.018165 11.730929 14.862092 18.133297  9.569294 17.651926
 14.061663 13.063938 11.023655 14.189661 11.679078]
[14.964113 14.185988 15.160908 12.126086 18.133297 15.160546 17.651926
 14.819087 11.771941 16.528748 15.815642 11.036083]
[13.877083  13.705082  13.16941   18.767145  15.07793   19.242002
 14.108535  14.5973835 20.461521  14.917007  13.168143  11.036083 ]
[17.805025 13.705082 13.16941  18.488708 17.970263 19.704533 14.974179
 13.998571 14.177486 16.025242 13.769693 16.886595]
[17.805025 13.95114  12.320407 16.194517 19.62

[18.486584 19.104986 15.486661 16.956554 20.438742 17.371262 15.401823
 12.268584 16.922583 13.391591 14.649695 12.85602 ]
[12.776629   9.03079   17.738583   8.350643  14.867083  17.371262
 15.401823  15.8641405 13.830268  14.552643  12.533114  14.425873 ]
[17.415224  13.705358  17.3514    16.631565  17.130325  13.06894
 15.09969   19.809446  15.0953045 18.305037  17.322994  14.425873 ]
[17.362083  16.290915  20.127937  16.861778  19.294537  15.024065
 16.96136   19.809446  15.0953045 13.403127  20.67833   16.721647 ]
[17.028294 16.290915 20.127937 13.884776 18.503428 13.62473  15.369268
 12.813808 13.426017 13.403127 16.937551 18.00377 ]
[20.204845 12.634465 13.90301  19.493975 18.107214 17.69921  17.961937
 16.780666 11.48519  17.070719 16.791689 14.621915]
[14.029097 16.550394 19.14182  19.493975 18.107214 15.870628 16.440342
 17.1287   11.48519  19.609694 17.022776 15.859439]
[14.029097 17.232805 12.003433 14.954863 15.193466 15.870628 16.440342
 15.679266 11.48519  14.754557 16.34

KeyboardInterrupt: 

In [5]:
for i in range(100):
    paths = glob.glob(f"{BLENDER_MEMBANK_ROOT}/**/obs_per_sec.npy", recursive=True)
    obs_per_sec = np.array([np.load(p)[0] for p in paths])
    print(obs_per_sec)
    time.sleep(60*1)

[]
[]
[14.222765 12.635613 12.003393 12.005242 11.759121 11.227034 14.356265
 10.951397 12.861022 15.181117 15.924361 12.548165]
[15.529956  15.142123  14.404691  13.045792  15.051752  15.7770815
 15.549948  15.682901  12.004563  16.348944  16.3124    13.120256 ]
[12.8857765 16.480331  15.388641  13.045792  15.051752  15.7770815
 14.70313   16.769428  19.796118  17.27344   12.586374  16.85441  ]


KeyboardInterrupt: 

In [5]:
set_should_stop(True)

In [3]:
set_lr(3e-4)

In [8]:
import numpy as np

In [9]:
a = np.array([1,2,3,4,1,2,3])

In [11]:
np.where(a==1)[0]

array([0, 4])

In [12]:
max(np.nan, 0)

nan

In [13]:
np.clip(12, 10,11)

11